In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2
import statsmodels.api as sm
import re
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Load environment variables
import os
from dotenv import load_dotenv
load_dotenv()


# Set up plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("tab10")

In [ ]:
#Get the database URL from environment variables
DATABASE_URL = os.getenv('DATABASE_URL')

# Create a connection to the SQLite database
conn = psycopg2.connect(DATABASE_URL)

# Define the SQL query to get comprehensive dataset for analysis
all_data_query = """
SELECT 
    p.doi,
    p.figure_number,
    s.sandstone_name,
    dp.p_mpa,
    dp.q_mpa
FROM data_points dp
JOIN sandstones s ON dp.sandstone_id = s.id
JOIN plots p ON s.plot_id = p.id
ORDER BY p.doi, s.sandstone_name, dp.p_mpa
"""

In [ ]:
# Convert the SQL query result into a DataFrame
df_all_data = pd.read_sql_query(all_data_query, conn)

# Display the DataFrame 
df_all_data

In [ ]:
# Filter for Rutter and Glover Figure 8 data
rut_glov_fig8 = df_all_data[df_all_data['doi'] == '10.1016/j.jsg.2012.08.014']

# Filter for friction data
friction_data = rut_glov_fig8[rut_glov_fig8['sandstone_name'] == 'Porous sandstone friction data']

# Remove Serpentinite data
rut_glov_fig8 = rut_glov_fig8[(rut_glov_fig8['sandstone_name'] != 'Serpentinite (20%)') & (rut_glov_fig8['sandstone_name'] != 'Porous sandstone friction data')]

## Recreating Rutter and Glover (2012) Figure 8

In [ ]:
# --- Sandstone regression ---
X = sm.add_constant(rut_glov_fig8['p_mpa'])
y = rut_glov_fig8['q_mpa']
model = sm.OLS(y, X).fit()
slope_sandstone = model.params['p_mpa']

# --- Friction regression ---
friction_X = sm.add_constant(friction_data['p_mpa'])
friction_y = friction_data['q_mpa']
friction_model = sm.OLS(friction_y, friction_X).fit()
slope_friction = friction_model.params['p_mpa']

# --- Combined plot ---
plt.figure(figsize=(10, 6))

# Sandstone data + regression
sns.scatterplot(data=rut_glov_fig8, x='p_mpa', y='q_mpa', hue='sandstone_name', alpha=0.7)
sns.regplot(data=rut_glov_fig8, x='p_mpa', y='q_mpa', scatter=False,
            color='black', ci=None, line_kws={'linewidth': 1.0, 'linestyle': '--'},
            label=f"Critical State Slope ({slope_sandstone:.2f})")

# Friction data + regression
sns.scatterplot(data=friction_data, x='p_mpa', y='q_mpa', color='yellow', s=20, edgecolor='black', label='Friction Data')
sns.regplot(data=friction_data, x='p_mpa', y='q_mpa', scatter=False,
            color='black', ci=None, line_kws={'linewidth': 0.5, 'linestyle': '-'},
            label=f"Friction Slope ({slope_friction:.2f})")

plt.xlabel('P (MPa)', fontsize=10)
plt.ylabel('Q (MPa)', fontsize=10)
plt.xlim(0, 1000)
plt.ylim(0, 1000)
plt.title('Digitised Rutter & Glover (2012) Fig. 8 Critical State and Friction Data')
plt.legend(loc='lower right', fontsize=9)
plt.grid(False)

plt.tight_layout()
plt.savefig("stats_plots/Rutter_Glover_2012_Fig8_Combined.png", dpi=600, bbox_inches='tight')
plt.show()

# Print regression summaries
print("OLS Regression Results for Rutter & Glover Figure 8 Sandstone Data")
print(model.summary())
print("\nOLS Regression Results for Rutter & Glover Figure 8 Friction Data")
print(friction_model.summary())


## Include Wong & Baud, 2012. Figures 5a,b and 6a,b (Limestones)

In [ ]:
# Wong and Baud Data
wong_baud = df_all_data[df_all_data['doi'] == '10.1016/j.jsg.2012.07.010']

# Combine (Wong and Baud) sandstone data with (Rutter and Glover) sandstone data
wong_baud_fig5 = wong_baud[wong_baud['figure_number'].isin(['5a', '5b'])]

rutter_glover_wong_baud = pd.concat([rut_glov_fig8, wong_baud_fig5], ignore_index=True)

# Wong and Baud Limestone Data
wong_baud_limestone = wong_baud[wong_baud['figure_number'].isin(['6a', '6b'])]

In [ ]:
# Create a regression plot for new data including Wong and Baud, 2012 data

# Sort the sandstone names for consistent ordering
sandstones = sorted(rutter_glover_wong_baud['sandstone_name'].unique())
n_groups = len(sandstones)

# Pairing colors and markers
colors = sns.color_palette("colorblind", n_colors=min(n_groups, 8))
markers = ['o', 's', 'D', '^', 'v', 'P', 'X', '<', '>'] 

fig, ax = plt.subplots(figsize=(10, 6))

# Plot each group separately
for i, name in enumerate(sandstones):
    group = rutter_glover_wong_baud[rutter_glover_wong_baud['sandstone_name'] == name]
    ax.scatter(
        group['p_mpa'], group['q_mpa'],
        label=name,
        color=colors[i % len(colors)],
        marker=markers[i % len(markers)],
        edgecolor='black',
        linewidth=0.4,
        s=40,
        alpha=0.8
    )

# Regression line
sns.regplot(
    data=rutter_glover_wong_baud, x='p_mpa', y='q_mpa',
    scatter=False, ci=None,
    line_kws={'color': 'black', 'linewidth': 1.0, 'linestyle': '--'},
    ax=ax
)

ax.set_xlabel('P (MPa)', fontsize=10)
ax.set_ylabel('Q (MPa)', fontsize=10)
ax.set_xlim(0, 1000)
ax.set_ylim(0, 1000)

# Legend
ax.legend(
    bbox_to_anchor=(1.02, 1), loc='upper left',
    fontsize=7, title='Sandstones',
    title_fontsize=8, frameon=False, ncol=1
)

plt.title('Regression: Rutter & Glover (2012) Fig 8 + Wong & Baud (2012) Fig 5a,5b', fontsize=11, pad=15)
plt.tight_layout()
plt.savefig("stats_plots/RutterGlover, 2012_Fig8 & WongBaud_2012 Fig5a,b.png", dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
# --- Rutter & Glover and Wong & Baud 2012 regression ---
X_wb = rutter_glover_wong_baud['p_mpa']
y_wb = rutter_glover_wong_baud['q_mpa']
X_const = sm.add_constant(X_wb)
wb_model = sm.OLS(y_wb, X_const).fit()
slope_wb = wb_model.params['p_mpa']

# Calculate additional metrics
y_pred = wb_model.predict(X_const)
r2 = r2_score(y_wb, y_pred)
mape = np.mean(np.abs((y_wb - y_pred) / np.where(y_wb == 0, 1e-10, y_wb))) * 100


# --- Combined plot ---
fig, ax = plt.subplots(figsize=(10, 6))

# Plot Rutter & Glover sandstone groups
sandstones = sorted(rutter_glover_wong_baud['sandstone_name'].unique())
colors = sns.color_palette("colorblind", n_colors=min(len(sandstones), 8))
markers = ['o', 's', 'D', '^', 'v', 'P', 'X', '<', '>']

for i, name in enumerate(sandstones):
    group = rutter_glover_wong_baud[rutter_glover_wong_baud['sandstone_name'] == name]
    ax.scatter(
        group['p_mpa'], group['q_mpa'],
        label=name,
        color=colors[i % len(colors)],
        marker=markers[i % len(markers)],
        edgecolor='black',
        linewidth=0.4,
        s=40,
        alpha=0.8
    )

# Regression line for Wong & Baud combined dataset
sns.regplot(
    data=rutter_glover_wong_baud, x='p_mpa', y='q_mpa',
    scatter=False, ci=None,
    line_kws={'color': 'black', 'linewidth': 1.0, 'linestyle': '--'},
    ax=ax,
    label=f"Critical State Slope ({slope_wb:.2f})"
)

# Add friction data
ax.scatter(
    friction_data['p_mpa'], friction_data['q_mpa'],
    color='yellow', s=20, edgecolor='black', label='Friction Data'
)
sns.regplot(
    data=friction_data, x='p_mpa', y='q_mpa',
    scatter=False, ci=None,
    line_kws={'color': 'black', 'linewidth': 0.5, 'linestyle': '-'},
    ax=ax,
    label=f"Friction Slope ({slope_friction:.2f})"
)

# Axes and legend
ax.set_xlabel('P (MPa)', fontsize=10)
ax.set_ylabel('Q (MPa)', fontsize=10)
ax.set_xlim(0, 1000)
ax.set_ylim(0, 1000)

ax.legend(
    bbox_to_anchor=(1.02, 1), loc='upper left',
    fontsize=7, 
    title_fontsize=8, frameon=False, ncol=1
)
ax.grid(False)

plt.title(f'Critical State Data for \n Rutter-Glover (2012) Fig 8 and Wong-Baud (2012) Fig. 5a,5b \n R²={r2:.3f}, MAPE={mape:.1f}%', fontsize=11, pad=15)
plt.tight_layout()
plt.savefig("stats_plots/RutterGlover_WongBaud_Figures_Combined.png", dpi=600, bbox_inches='tight')
plt.show()

# Print regression summaries
print("\nOLS Regression Results for Wong & Baud 2012 Sandstone Data")
print(wb_model.summary())


_____
## Can We Estimate Rock Strength Using Porosity and Grain Size?

In [ ]:
# Grain size of sandstones in Rutter and Glover, 2012
grain_sizes = {
    'Darley Dale (13%)': 171,
    'Berea wet (18 & 21%)' : 170,
    'Hollington (25%)': 230,
    'Penrith (28%)': 129.6
}

# Extract Porosity of the Sandstones
def extract_porosity(s):
    """
    Extracts the porosity percentage from a string formatted like "(x%)"
    """
    matches = re.findall(r"(\d+(?:\.\d+)?)%", s)
    return float(matches[0]) if matches else None
    
# Apply the function to the 'sandstone_name' column
rut_glov_fig8['porosity'] = rut_glov_fig8['sandstone_name'].apply(extract_porosity)

# Create a DataFrame to include porosity and grain size
rutter_glover_with_por_gsize = pd.DataFrame({
    'Sandstone': rut_glov_fig8['sandstone_name'],
    'P (MPa)': rut_glov_fig8['p_mpa'],
    'Q (MPa)': rut_glov_fig8['q_mpa'],
    'Porosity (%)': rut_glov_fig8['porosity'],
    'Grain Size (mm)': [grain_sizes.get(name, None) for name in rut_glov_fig8['sandstone_name']]
})
# Display the porosity and grain size DataFrame
rutter_glover_with_por_gsize.head()

In [ ]:
# Define function to calculate the hydrostatic pressure P*
def hydrostatic_pressure(porosity, grain_size_µm):
    """
    Calculate the hydrostatic pressure P* based on porosity (%) and grain size (µm).

    Equation:
    log10(P*) [MPa] = 0.625 - 1.064 * log10(porosity_decimal * grain_size_mm)

    Parameters
    ----------
    porosity : float
        Porosity in percent (%).
    grain_size_um : float
        Grain size in micrometres (µm).

    Returns
    -------
    float
        Hydrostatic pressure P* in MPa.
    """
    if porosity is None or grain_size_µm is None:
        return None

    porosity_decimal = porosity / 100
    grain_size_mm = grain_size_µm / 1000.0   # convert µm → mm

    fr_mm = porosity_decimal * grain_size_mm
    log10_P = 0.625 - 1.064 * np.log10(fr_mm)
    return 10 ** log10_P


# Apply the hydrostatic pressure function to the DataFrame
rutter_glover_with_por_gsize['P* (MPa)'] = rutter_glover_with_por_gsize.apply(
    lambda row: hydrostatic_pressure(row['Porosity (%)'], row['Grain Size (mm)']), axis=1
)
# Display the DataFrame with hydrostatic pressure
rutter_glover_with_por_gsize.tail()


In [ ]:
# Normalise P and Q values by diving by the hydrostatic pressure P*
rutter_glover_with_por_gsize['P_norm'] = rutter_glover_with_por_gsize['P (MPa)'] / rutter_glover_with_por_gsize['P* (MPa)']
rutter_glover_with_por_gsize['Q_norm'] = rutter_glover_with_por_gsize['Q (MPa)'] / rutter_glover_with_por_gsize['P* (MPa)']

# Create a regression plot for the normalised data
plt.figure(figsize=(10, 6))
sns.scatterplot(data=rutter_glover_with_por_gsize, x='P_norm', y='Q_norm', hue='Sandstone')
sns.regplot(data=rutter_glover_with_por_gsize, x='P_norm', y='Q_norm', scatter=False, 
            color='black', ci=None, label='Regression Line')
plt.xlabel('P_norm (P / P*)', fontsize=10)
plt.ylabel('Q_norm (Q / P*)', fontsize=10)
plt.title('Normalised Regression Plot for Rutter and Glover, 2012 Sandstone Data')
plt.legend(loc='lower right', title='Sandstones')
plt.savefig("stats_plots/Rutter_Glover_2012_Normalised_P_Q.png",
           dpi=600, bbox_inches='tight')
plt.show()
# OLS Regression for normalised data
X_norm = rutter_glover_with_por_gsize['P_norm']
y_norm = rutter_glover_with_por_gsize['Q_norm']
X_norm = sm.add_constant(X_norm)  # Adds a constant term to the predictor
model_norm = sm.OLS(y_norm, X_norm).fit()
print(f"OLS Regression Results for Normalised Rutter and Glover, 2012 Sandstone Data \n")
print(model_norm.summary())


_______
## Include Additional Sandstones from Wong & Baud

In [ ]:
# Grain size of sandstones in Rutter and Glover, 2012
grain_sizes_wong_baud = {
    'Adamswiller (22.6%), Wong et al. (1997)': 90,
    'Bentheim (22.8%), Baud et al. (2006)' : 105,
    'Berea (21%), Baud et al. (2006)': 130,
    'Bleurwiller (25.2%), Tembe et al. (2008), Fortin et al. (2006)': 112,
    'Boise (35%), Baud et al. (2000b)': 280,
    'Darley Dale (13%), Baud et al. (2004)': 170,
    'Diemelstadt (24.3%), Tembe et al. (2008)': 80,
    'Kayenta (21%), Wong et al. (1997)': 150,
    'Rothbach 90 (20%), Louis et al. (2009)': 230,
    'Darley Dale (13%)': 171,
    'Berea wet (18 & 21%)' : 170,
    'Hollington (25%)': 230,
    'Penrith (28%)': 129.6
}
    
# Apply the function to the 'sandstone_name' column
rutter_glover_wong_baud['porosity'] = rutter_glover_wong_baud['sandstone_name'].apply(extract_porosity)

# Create a DataFrame to include porosity and grain size
rutter_glover_wong_baud_with_por_gsize = pd.DataFrame({
    'Sandstone': rutter_glover_wong_baud['sandstone_name'],
    'P (MPa)': rutter_glover_wong_baud['p_mpa'],
    'Q (MPa)': rutter_glover_wong_baud['q_mpa'],
    'Porosity (%)': rutter_glover_wong_baud['porosity'],
    'Grain Size (mm)': [grain_sizes_wong_baud.get(name, None) for name in rutter_glover_wong_baud['sandstone_name']]
})
# Display the porosity and grain size DataFrame
rutter_glover_wong_baud_with_por_gsize.head()

In [ ]:
# Apply the hydrostatic pressure function to the DataFrame
rutter_glover_wong_baud_with_por_gsize['P* (MPa)'] = rutter_glover_wong_baud_with_por_gsize.apply(
    lambda row: hydrostatic_pressure(row['Porosity (%)'], row['Grain Size (mm)']), axis=1
)

# Normalise P and Q values by diving by the hydrostatic pressure P*
rutter_glover_wong_baud_with_por_gsize['P_norm'] = rutter_glover_wong_baud_with_por_gsize['P (MPa)'] / rutter_glover_wong_baud_with_por_gsize['P* (MPa)']
rutter_glover_wong_baud_with_por_gsize['Q_norm'] = rutter_glover_wong_baud_with_por_gsize['Q (MPa)'] / rutter_glover_wong_baud_with_por_gsize['P* (MPa)']

In [ ]:
# =====================================================
# Calculate Performance Metrics
# =====================================================

# Prepare data for regression
df = rutter_glover_wong_baud_with_por_gsize.copy()

X_norm = df['P_norm'].values
y_norm = df['Q_norm'].values

# Fit regression model
X_norm_const = sm.add_constant(X_norm)
model_norm_wong_baud = sm.OLS(y_norm, X_norm_const).fit()

# Calculate additional metrics
y_pred = model_norm_wong_baud.predict(X_norm_const)
r2 = r2_score(y_norm, y_pred)
mae = mean_absolute_error(y_norm, y_pred)
rmse = np.sqrt(mean_squared_error(y_norm, y_pred))
mape = np.mean(np.abs((y_norm - y_pred) / np.where(y_norm == 0, 1e-10, y_norm))) * 100

# Get regression parameters (handle both named and indexed access)
try:
    # Try named access first
    slope = model_norm_wong_baud.params['P_norm']
    intercept = model_norm_wong_baud.params['const']
    p_value = model_norm_wong_baud.pvalues['P_norm']
except (KeyError, IndexError):
    # Fall back to indexed access [intercept, slope]
    slope = model_norm_wong_baud.params[1]
    intercept = model_norm_wong_baud.params[0]
    p_value = model_norm_wong_baud.pvalues[1]
n_points = len(y_norm)

# =====================================================
# Create Plot
# =====================================================

plt.figure(figsize=(10, 6))

# Main scatter plot with colors by sandstone type
sns.scatterplot(data=df, x='P_norm', y='Q_norm', hue='Sandstone', 
                alpha=0.7, s=50, edgecolor='black', linewidth=0.3)

# Regression line
sns.regplot(data=df, x='P_norm', y='Q_norm', scatter=False, 
            color='black', ci=None, line_kws={'linewidth': 1, 'linestyle': '-'})

# =====================================================
# Styling and Labels
# =====================================================

plt.xlabel('P_normalized (P / P*)', fontsize=12, fontweight='bold')
plt.ylabel('Q_normalized (Q / P*)', fontsize=12, fontweight='bold')
plt.grid(False)

# Enhanced title with key results
plt.title(f'Normalized Regression Plot for Rutter-Glover (2012) + Wong-Baud (2012)\n'
         f'Universal Relationship: R² = {r2:.3f}, MAPE = {mape:.1f}%, Slope = {slope:.2f}', 
         fontsize=12, pad=20)

# Enhanced legend
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left',
          fontsize=8, title='Sandstone Types',
          title_fontsize=10, frameon=True, 
          fancybox=True, shadow=True, ncol=1)

# Add equation text
equation_text = f'Q_norm = {slope:.3f} x P_norm + {intercept:.3f}'
plt.text(0.02, 0.02, equation_text, transform=plt.gca().transAxes,
         fontsize=11, fontweight='bold',
         bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.8))

# Tight layout and save
plt.tight_layout()
plt.savefig("stats_plots/RutterGlover_WongBaud_Normalised_P_Q_Enhanced.png",
           dpi=600, bbox_inches='tight')
plt.show()

print(f"\nPERFORMANCE METRICS:")
print(f"  Mean Absolute Error:      {mae:.4f}")
print(f"  Root Mean Square Error:   {rmse:.4f}")
print(f"  Mean Absolute Percentage Error:    {mape:.2f}%")

print(f"\nREGRESSION EQUATION:")
print(f"  Q_normalized = {slope:.4f} × P_normalized + {intercept:.4f}")

# Statistical summary
print(f"\n" + "="*80)
print("DETAILED STATISTICAL SUMMARY")
print("="*80)
print(model_norm_wong_baud.summary())

________

# Different Porosoity Plots

In [ ]:
# Split data based on median porosity

median_porosity = rutter_glover_wong_baud_with_por_gsize['Porosity (%)'].median()

# Dataframes for porosity below and above median
below_median = rutter_glover_wong_baud_with_por_gsize[rutter_glover_wong_baud_with_por_gsize['Porosity (%)'] < median_porosity]
above_median = rutter_glover_wong_baud_with_por_gsize[rutter_glover_wong_baud_with_por_gsize['Porosity (%)'] >= median_porosity]

In [ ]:
# Sort the sandstone names for consistent ordering
sandstones_below = sorted(below_median['Sandstone'].unique())
n_groups = len(sandstones_below)

# Pairing colors and markers
colors = sns.color_palette("colorblind", n_colors=min(n_groups, 5))
markers = ['o', 's', 'D', '^', 'v', 'P', 'X', '<', '>'] 

fig, ax = plt.subplots(figsize=(10, 6))
for i, name in enumerate(sandstones_below):
    group = below_median[below_median['Sandstone'] == name]
    ax.scatter(
        group['P (MPa)'], group['Q (MPa)'],
        label=name,
        color=colors[i % len(colors)],
        marker=markers[i % len(markers)],
        edgecolor='black',
        linewidth=0.4,
        s=40,
        alpha=0.8
    )
sns.regplot(
    data=below_median, x='P (MPa)', y='Q (MPa)',
    scatter=False, ci=None,
    line_kws={'color': 'black', 'linewidth': 1.0, 'linestyle': '--'},
    ax=ax
)
ax.set_xlabel('P (MPa)', fontsize=10)
ax.set_ylabel('Q (MPa)', fontsize=10)
ax.set_xlim(0, 1000)
ax.set_ylim(0, 1000)

# Legend
ax.legend(
    bbox_to_anchor=(1.02, 1), loc='upper left',
    fontsize=7, title='Sandstones',
    title_fontsize=8, frameon=False, ncol=1
)

plt.title(f'Regression: Sandstones with porosity less than median value:{median_porosity}%', fontsize=11, pad=15)
plt.tight_layout()
plt.savefig("stats_plots/porosity less than 22.8.png", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
# OLS Regression for below median porosity
X_below = below_median['P (MPa)']
y_below = below_median['Q (MPa)']
X_below = sm.add_constant(X_below)  # Adds a constant term to the predictor
model_below = sm.OLS(y_below, X_below).fit()
print(f"Regression Stats for Sandstones less than 22.8: \n{model_below.summary()}")

In [ ]:
# Sort the sandstone names for consistent ordering
sandstones_above = sorted(above_median['Sandstone'].unique())
n_groups = len(sandstones_above)

# Pairing colors and markers
colors = sns.color_palette("colorblind", n_colors=min(n_groups, 5))
markers = ['o', 's', 'D', '^', 'v', 'P', 'X', '<', '>'] 

fig, ax = plt.subplots(figsize=(10, 6))
for i, name in enumerate(sandstones_above):
    group = above_median[above_median['Sandstone'] == name]
    ax.scatter(
        group['P (MPa)'], group['Q (MPa)'],
        label=name,
        color=colors[i % len(colors)],
        marker=markers[i % len(markers)],
        edgecolor='black',
        linewidth=0.4,
        s=40,
        alpha=0.8
    )
sns.regplot(
    data=above_median, x='P (MPa)', y='Q (MPa)',
    scatter=False, ci=None,
    line_kws={'color': 'black', 'linewidth': 1.0, 'linestyle': '--'},
    ax=ax
)
ax.set_xlabel('P (MPa)', fontsize=10)
ax.set_ylabel('Q (MPa)', fontsize=10)
ax.set_xlim(0, 1000)
ax.set_ylim(0, 1000)

# Legend
ax.legend(
    bbox_to_anchor=(1.02, 1), loc='upper left',
    fontsize=7, title='Sandstones',
    title_fontsize=8, frameon=False, ncol=1
)

plt.title(f'Regression: Sandstones with porosity higher than median value:{median_porosity}%', fontsize=11, pad=15)
plt.tight_layout()
plt.savefig("stats_plots/porosity more than 22.8.png", dpi=600, bbox_inches='tight')
plt.show()



In [ ]:
# OLS Regression for above median porosity
X_above = above_median['P (MPa)']
y_above = above_median['Q (MPa)']
X_above = sm.add_constant(X_above)  # Adds a constant term to the predictor
model_above = sm.OLS(y_above, X_above).fit()
print(f"Regression Stats for Sandstones higher than 22.8: \n{model_above.summary()}")

__________

## Limestone Critical State Data

In [ ]:
# Create a regression plot for limestones
plt.figure(figsize=(10, 6))
sns.scatterplot(data=wong_baud_limestone, x='p_mpa', y='q_mpa', hue='sandstone_name')
sns.regplot(data=wong_baud_limestone, x='p_mpa', y='q_mpa', scatter=False, 
            ci=None, line_kws={'color': 'black', 'linewidth': 1.0, 'linestyle': '--'})
plt.xlim(0, 500)
plt.ylim(0, 500)
plt.xlabel('P (MPa)')
plt.ylabel('Q (MPa)')
plt.legend(loc='lower right', title='Limestones')
plt.title('Regression Plot for Limestones - Wong & Baud 2012 Figure 6a,6b')
plt.savefig("stats_plots/Limestones WongBaud_2012 Fig6a,b.png", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
# OLS Regression for limestones
X_limestone = wong_baud_limestone['p_mpa']
y_limestone = wong_baud_limestone['q_mpa']
X_limestone = sm.add_constant(X_limestone)  # Adds a constant term to the
model_limestone = sm.OLS(y_limestone, X_limestone).fit()
print(f" Wong & Baud Fig 6a,b \n{model_limestone.summary()}")

## Limestone Friction Data - Byerlee, 1978. Figure 4

In [ ]:
# filter for limestone friction data
limestone_friction_data = df_all_data[df_all_data['doi'] == '10.1007/BF00876528']

In [ ]:
# Create a regression plot for the friction data
plt.figure(figsize=(10, 6))
sns.scatterplot(data=limestone_friction_data, x='p_mpa', y='q_mpa')
sns.regplot(data=limestone_friction_data, x='p_mpa', y='q_mpa', scatter=False, 
            color='black', ci=None, label='Regression Line', 
            line_kws={'linewidth': 1.0, 'linestyle': '--'})
plt.xlim(0, 150)
plt.ylim(0, 50)
plt.xlabel('P (MPa)', fontsize=10)
plt.ylabel('Q (MPa)', fontsize=10)
plt.title('Regression Plot for Byerlee, 1978 Figure 4 Limestone Friction Data')
plt.savefig("stats_plots/Byerlee_1978_Fig4_FrictionData.png", dpi=600, bbox_inches='tight')
plt.show()

# OLS for friction data
friction_X = limestone_friction_data['p_mpa']
friction_y = limestone_friction_data['q_mpa']
friction_X = sm.add_constant(friction_X)  # Adds a constant term to the predictor
limestone_friction_model = sm.OLS(friction_y, friction_X).fit()
print(f"OLS Regression Results for Byerlee, 1978 Figure 4 Limestone Friction Data \n")
print(limestone_friction_model.summary())